In [1]:
import pandas as pd  # таблицы

# токенизация
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import download
from nltk.corpus import stopwords

download("punkt")
download("stopwords")

from textblob import TextBlob # сентимент-анализ

dir = "D:\HSE\Pump_Dump\\"  # папка проекта

# список символов для удаления
with open(dir + "clear_text.txt", encoding="utf8") as tmp_txt:
    clear_sym = tmp_txt.read().split("\n")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ashku\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ashku\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# ЭТОТ КОД ЛУЧШЕ ПЕРЕМЕСТИТЬ В КЛАСС 
# загрузка фрейма и создание списка тикеров с альтернативными названиями
tickers_df = pd.read_csv(
    dir + "market\\moex_my.csv", encoding="utf-8", sep="|", keep_default_na=False
)
tickers = list()
# в каждой строке собираем значения всех столбцов
for i in range(0, len(tickers_df)):
    tickers_temp = list()
    tickers_temp += tickers_df.iloc[i]["FULL_NAME"].split(",")
    tickers_temp += tickers_df.iloc[i]["SHORT_NAME"].split(",")
    tickers_temp += tickers_df.iloc[i]["NORM_NAME"].split(",")
    tickers_temp += tickers_df.iloc[i]["MORPH_NAMES"].split(",")
    tickers_temp += tickers_df.iloc[i]["MY_NAMES"].split(",")
    # добавляем в сет для удаления дублей и фильтруем для удаления пустых значений
    tickers_temp = set(tickers_temp)
    tickers_temp = list(filter(None, tickers_temp))
    # тикеры ставим в начало списка
    tickers_temp = tickers_df.iloc[i]["TICKERS"].split(",") + tickers_temp
    tickers.append(tickers_temp)


In [3]:
# токенизация. Параметр - любой текст. Возвращает список токенов текста
def clear_text(text):
    text = text.lower()
    text = word_tokenize(text)
    text_without_punkt = []
    for word in text:
        if word[0].isalpha():
            text_without_punkt.append(word)
    # в стоп-словах stop-words NLTK есть кейсы, которые могут влиять на восприятие текста (не исключено -> исключено)
    stop_words = stopwords.words("russian")
    # добавляем свои стоп-слова из файла clear_text.txt
    stop_words = stop_words + clear_sym
    clean_text = []
    for word in text_without_punkt:
        if word not in stop_words:
            clean_text.append(word)

    return clean_text

In [4]:
# тикеризация. Параметр - список токенов текста. Возвращает список тикеров текста
def tickers_find(tokens_list):
    # список токенов преобразуем в set для удаления дублей токенов
    tokens_list = set(tokens_list)
    tickers_list = set()
    temp_text = ""
    for token in tokens_list:
        # ищем в сете с тикерами и альтернативными названиями TICKERS|SHORT_NAME|NORM_NAME|MORPH_NAMES|MY_NAMES
        find_index = list(tickers_df[tickers_df.isin([token]).any(axis=1)].index.values)
        if len(find_index) > 0:
            temp_text = (
                tickers_df.loc[[int(find_index[0])]]["TICKERS"].values[0].split(",")[0]
            )
        # если по единичным значениям столбцов не найдено, то берем полный список альтернативных названий
        else:
            for ticker in tickers:
                if token in ticker:
                    temp_text = ticker[0]
        if len(temp_text) > 0:
            tickers_list.add(temp_text)
    # print (tokens_list,list(tickers_list),sep="\n")
    return list(tickers_list)

# для тестов по произвольным строкам
# print(tickers_find(["цифры","сургутнефтегаз","irao","лукойл",]))

In [5]:
# считывание и нормализация сырых данных канала. Параметр - название ТГ-канала (загруженный на диск CSV). Возвращает  фрейм с данными канала
def channel_post(channel):
    channel_path = dir + "TG\\" + channel + ".csv"
    # загрузка файла
    df = pd.read_csv(
        channel_path,
        encoding="utf-8",
        sep="|",
        keep_default_na=False,
    )
    # преобразуем поле с датой новости в тип ДАТА-ВРЕМЯ
    df["NEWS_DATE"] = pd.to_datetime(df["NEWS_DATE"], dayfirst=True, errors="ignore")
    # создаем поля для статистики по датам Номер недели
    df = df.assign(NEWS_YEAR=df["NEWS_DATE"].dt.isocalendar().year)
    df = df.assign(NEWS_WEEK=df["NEWS_DATE"].dt.isocalendar().week)
    # создаем поля для ключевых слов, тикеров и сентимента
    df = df.assign(WORDS="")
    df = df.assign(TICKERS="")
    df = df.assign(MOOD="")
    # в некоторых новостях встречаются дубли текстов, удаляем их
    df.drop_duplicates(subset="NEWS_TEXT", keep="first", inplace=True)
    return df

In [6]:
# сентимент анализ
def mood_text(text):
    # Используем TextBlob для анализа тональности
    analysis = TextBlob(text)
    # Оцениваем настроение текста
    sentiment = analysis.sentiment.polarity
    # The sentiment property returns a namedtuple of the form Sentiment(polarity, subjectivity).
    # The polarity score is a float within the range [-1.0, 1.0].
    # The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.
    # print(analysis.sentiment)
    if sentiment > 0:
        sentiment_label = "positive"
    elif sentiment < 0:
        sentiment_label = "negative"
    else:
        sentiment_label = "neutral"

    return sentiment_label

In [7]:
# тикеризация и токенизация текста
channels = pd.read_csv(dir + "TG\\" + "!channels.csv", delimiter="|")
print("Будет обработано: ",len(channels))
k=1
# обрабатываем каждый канал
for channel in channels["CHANNEL"].values:
    print(k, channel)
    df = channel_post(channel)
    # для каждой новости создаем список ключевых слов и выделяем тикеры
    for i in range(0, len(df)):
        news_text = df.iloc[i]["NEWS_TEXT"]
        words_text = clear_text(news_text)
        df["WORDS"].values[i] = ",".join(words_text)
        df["TICKERS"].values[i] = ",".join(tickers_find(words_text))
        df["MOOD"].values[i] = mood_text(news_text)

    # обновленный фрейм записываем на диск с признаком _post (пост-обработка)
    df.to_csv(
        dir + "TG\\" + channel + "_post.csv",
        index=False,
        encoding="utf-8",
        sep="|",
    )
    k+=1

Будет обработано:  30
1 signals_moex
2 sonbuffet
3 smfanton
4 investoksana
5 kapitalopedia
6 russia_companies
7 dohod_investment
8 sgcapital
9 ruforecasts
10 anoniminvest
11 iporobinhood
12 mamonova_finance
13 dohod
14 if_stocks
15 lemonfortea
16 div_invest
17 tinkoff_invest_official
18 finamalert
19 bcs_express
20 omyinvestments
21 alfa_investments
22 fm_invest
23 cbrstocks
24 AK47pfl
25 fineconomics
26 invest_or_lost
27 profitking
28 bizlike
29 spydell_finance
30 divForever


In [ ]:
#для тестов тикеризации
# txt = "Возможно, «Газпром» выплатит дивиденды за 2023 год, не исключено, что с дивдоходностью около 10%. Но это все равно ниже безрисковой ставки и того же «Сбера», у которого бизнес более привлекательный – Алор Брокер У цен на газ предпосылок для роста пока нет, скорее, консолидация в районе $300 за 1 тыс. куб. м. Это умеренно негативно для «Газпрома», так как компания зависит и от спроса Европы на газ, и от цен. Мы считаем, что Китай не сможет в полной мере покрыть выпавшие европейские объемы. По акциям «Газпрома» сохраняем рекомендацию «держать» и ориентир 190 рублей за акцию – ПСБ При отсутствии признаков дальнейшего ускорения роста цен до конца января не исключаем возврат кривой ОФЗ к ноябрьским минимумам по доходности 11,6% по 5 летним ОФЗ и 11,7 11,75% по 10 летним – ПСБ Шансов на выход рынка из боковика вверх больше, чем на уход индекса МосБиржи ниже 3100 пунктов. В такой ситуации инвесторам стоит занимать выжидательную позицию, а спекулянтам – искать идеи во втором третьем эшелоне – Алор Брокер Причин для девальвации российской валюты в ближайшее время мы не видим. Рубль по прежнему будет получать поддержку от огромных продаж валюты Банком России и низкого спроса импортеров на доллары и юани в начале года эшелоне – Алор Брокер Finam Alert это рыночные сигналы, идеи, торговые прогнозы|2024|3|возможно, газпром, выплатит, дивиденды, год, исключено, дивдоходностью, около, это, равно, ниже, безрисковой, ставки, сбера, которого, бизнес, привлекательный, алор, брокер, цен, газ, предпосылок, роста, пока, скорее, консолидация, районе, тыс, куб, м., это, умеренно, негативно, газпрома, компания, зависит, спроса, европы, газ, цен, считаем, китай, сможет, полной, мере, покрыть, выпавшие, европейские, объемы, акциям, газпрома, сохраняем, рекомендацию, держать, ориентир, рублей, акцию, псб, отсутствии, признаков, дальнейшего, ускорения, роста, цен, конца, января, исключаем, возврат, кривой, офз, ноябрьским, минимумам, доходности, летним, офз, летним, псб, шансов, выход, рынка, боковика, вверх, уход, индекса, мосбиржи, ниже, пунктов, ситуации, инвесторам, стоит, занимать, выжидательную, позицию, спекулянтам, искать, идеи, втором, третьем, эшелоне, алор, брокер, причин, девальвации, российской, валюты, ближайшее, время, видим, рубль, прежнему, получать, поддержку, огромных, продаж, валюты, банком, россии, низкого, спроса, импортеров, доллары, юани, начале, года, эшелоне, алор, брокер, finam, alert, это, рыночные, сигналы, идеи, торговые, прогнозы"
# print(clear_text(txt))
# print(tickers_find(clear_text(txt)))